In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf
import csv

from sklearn.metrics import accuracy_score

# Common imports
import numpy as np
import os

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed
                  )
from sklearn.preprocessing import StandardScaler

#PREPARING FOR TENSORBOARD:
from datetime import datetime
#FOR DIFFERENT MODELS MAKE DIFFERENT LOG NAME!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/vege_1_1_mid_pool-{}/".format(root_logdir, now)

In [2]:
print ('DATA PREPARATION')
vege_csv = csv.reader(open('merged1301_clean.csv', newline=''), delimiter=' ', quotechar='|')

vege_array = []

for row in vege_csv:
	red_array = []
	items = row[0].split(';')
	for i in items:
		red_array.append(i)

	vege_array.append(red_array)

klase = []
values = []

vege_array_fix = vege_array[:-4]

print ('length of the fixed array is ', len (vege_array_fix))

for r in vege_array_fix:
    value_row = []
    i = 0
    while i < len(r):
        if i == 0:
#             klase.append(float(r[i]))
            
            if r[i] == '0':
                klase.append(0)
            if r[i] == '1':
                klase.append(1)
            if r[i] == '2':
                klase.append(2)
        else:
            value_row.append(float(r[i]))
        i = i + 1
    values.append(value_row)

print ('test : ', values[10000])
print ('number of value arrays is: ', len(values), ' and number of classes is: ', len(klase))

DATA PREPARATION
length of the fixed array is  3651826
test :  [305.0, 162.0, 188.0, 184.0, 114.0, 438.0, 738.0, 698.0]
number of value arrays is:  3651826  and number of classes is:  3651826


In [3]:
X_train, X_test, y_train, y_test = train_test_split(values, klase, test_size=0.20, random_state=42)

scaler = StandardScaler()
X_train = np.array(X_train) #.astype('float32')
X_train = scaler.fit_transform(X_train.astype(np.float64))

In [4]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [5]:
n_channels = 8
n_outputs = 3
height = 1
width = 1

#not sure if this is right way to define inputs if I would like to make
#pixel based classification model
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, n_channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [6]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
validlen = int(len(X_train) * 0.1)

X_valid = X_train[:validlen]
X_train = X_train[validlen:]
y_valid = y_train[:validlen]
y_train = y_train[validlen:]
#valied length is used primarily for implementing early stopping

In [8]:
n_epochs = 500
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./simple_dnn_vege.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./simple_dnn_vege.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

TypeError: only integer scalar arrays can be converted to a scalar index

1. error iz prethodne ćelije:
TypeError: only integer scalar arrays can be converted to a scalar index
- TESTIRAJ NA 